https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

###Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')#data set was in HM's googledrive and imported it

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd #python data manipulation library
from sklearn.model_selection import train_test_split #splits data into training and testing

import torch #new machine learning tools from PyTorch (a new model file extension)
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
df = pd.read_csv("/content/drive/My Drive/MLBootcamp/Week Three/Day 12/weatherAUS.csv") #tells computer to read the weather data

In [ ]:
df['RainToday'].replace({'No': 0, 'Yes': 1}, inplace = True) #this is weather data after running it
df['RainTomorrow'].replace({'No': 0, 'Yes': 1}, inplace = True) #one hot vector
df = df.dropna(how='any')
X = df[['Rainfall', 'Humidity3pm', 'RainToday', 'Pressure9am']] #features
y = df[['RainTomorrow']] #labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

###PyTorch Model

In [ ]:
X_train = torch.from_numpy(X_train.to_numpy()).float() #numpy (matrix#) representation of data
y_train = torch.squeeze(torch.from_numpy(y_train.to_numpy()).float())
X_test = torch.from_numpy(X_test.to_numpy()).float()
y_test = torch.squeeze(torch.from_numpy(y_test.to_numpy()).float())

In [ ]:
class Net(nn.Module): #this says it's class that takes a nearal net model. Base class for all nn models
  def __init__(self, n_features):
    super(Net, self).__init__() #I think self referes to the 4 starting feature.
    self.fc1 = nn.Linear(n_features, 5)#Number of features is 4, then 5, then 3, then 1
    self.fc2 = nn.Linear(5, 3) #5 nodes to 3 #nodes
    self.fc2point5 = nn.Linear(3, 3) 
    self.fc3 = nn.Linear(3, 2)#Here is where I'm adding my layer for the homework!!!!!!!!!!!!
    self.fc4 = nn.Linear(2, 1) #Now my layers go from 4 to 5 to 3 to 2 to 1 layer
  def forward(self, x): #these next three steps are the activation layers
    x = F.relu(self.fc1(x)) #does Relu Function for the first layer (max between 0 and f(c1))
    x = F.relu(self.fc2(x)) #does Relu Function for the second layer (max between 0 and f(c2))
    x = F.leaky_relu(self.fc2point5(x))
    x = F.relu(self.fc3(x)) #Here is where I added Relu to my new third layer for homework!!!!!
    return torch.sigmoid(self.fc4(x)) #does the sigmoid function (bewteen 0 and 1) for the fourth layer

In [ ]:
net = Net(X_train.shape[1]) #create a net that says we start with 4 features
print(net) #this prints the nets out and shows they go from 4 to 5 to 3 to 2 to 1

Net(
  (fc1): Linear(in_features=4, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=3, bias=True)
  (fc2point5): Linear(in_features=3, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=1, bias=True)
)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #cuda means it's running on CPU

In [ ]:
num_epochs = 700 #changed from 500 to 700
net = net.to(device) #need this if you are switching type of training device
criterion = nn.BCELoss().to(device) #loss function
optimizer = optim.Adadelta(net.parameters()) # Homework answer. this makes the learning step adaptive. This does the gradient descent. .001 is the default learning rate.I made it lower so model "skips more and hobbles less"

In [ ]:
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

In [ ]:
def calculate_accuracy(y_true, y_pred): #these functions calculate accuracy
  predicted = y_pred.ge(.5).view(-1) #set to 0 if < 0.5, 1 if >= 0.5
  return (y_true == predicted).sum().float() / len(y_true)

def round_tensor(t, decimal_places=3):
  return round(t.item(), decimal_places)

###Training

In [ ]:
for epoch in range(num_epochs):
    y_pred = net(X_train) ##important
    y_pred = torch.squeeze(y_pred)
    train_loss = criterion(y_pred, y_train) ##important
    if epoch % 100 == 0:
      train_acc = calculate_accuracy(y_train, y_pred) 
      y_test_pred = net(X_test) 
      y_test_pred = torch.squeeze(y_test_pred)
      test_loss = criterion(y_test_pred, y_test)
      test_acc = calculate_accuracy(y_test, y_test_pred)

      model_file = {'model': Net(4),
          'state_dict': net.state_dict(),
          'optimizer' : optimizer.state_dict()}

      torch.save(model_file, 'model%d.pth'% epoch)


      print(
f'''epoch {epoch}
Train set - loss: {round_tensor(train_loss)}, accuracy: {round_tensor(train_acc)}
Test  set - loss: {round_tensor(test_loss)}, accuracy: {round_tensor(test_acc)}
''')
    optimizer.zero_grad() ##important, set the gradient to 0
    train_loss.backward() ##important, backward propagation
    optimizer.step() ##important, updates the optimizer
    
    

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


epoch 0
Train set - loss: 0.632, accuracy: 0.78
Test  set - loss: 0.632, accuracy: 0.78

epoch 100
Train set - loss: 0.571, accuracy: 0.78
Test  set - loss: 0.571, accuracy: 0.78

epoch 200
Train set - loss: 0.539, accuracy: 0.78
Test  set - loss: 0.539, accuracy: 0.78

epoch 300
Train set - loss: 0.529, accuracy: 0.78
Test  set - loss: 0.528, accuracy: 0.78

epoch 400
Train set - loss: 0.527, accuracy: 0.78
Test  set - loss: 0.527, accuracy: 0.78

epoch 500
Train set - loss: 0.527, accuracy: 0.78
Test  set - loss: 0.527, accuracy: 0.78

epoch 600
Train set - loss: 0.527, accuracy: 0.78
Test  set - loss: 0.527, accuracy: 0.78



###Inference

Make sure if you train on a GPU, you run the model on a GPU.

In [ ]:
def load_model(filepath):
    model_path = torch.load(filepath)
    model = model_path['model']
    model.load_state_dict(model_path['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False

    model.eval() ##super important
    return model

model = load_model('model900.pth')

FileNotFoundError: ignored

In [ ]:
model.cuda() #if on GPU, or don't need this line

Net(
  (fc1): Linear(in_features=4, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=1, bias=True)
)

In [ ]:
output = model(X_test)

In [ ]:
y_pred = torch.squeeze(output)

In [ ]:
calculate_accuracy(y_test, y_pred)

tensor(0.7798, device='cuda:0')

In [ ]:
model.state_dict()

OrderedDict([('fc1.weight', tensor([[-0.2364, -0.0062, -0.5604,  0.1197],
                      [ 0.1397, -0.4747, -0.1354, -0.4556],
                      [-0.1248,  0.3450,  0.0423,  0.4136],
                      [-0.0800,  0.4399, -0.1816,  0.0870],
                      [ 0.0427, -0.3346, -0.4804, -0.4379]], device='cuda:0')),
             ('fc1.bias',
              tensor([ 0.0782, -0.3728,  0.2775, -0.0620,  0.1338], device='cuda:0')),
             ('fc2.weight',
              tensor([[ 0.0768, -0.1645, -0.0829, -0.0797,  0.2555],
                      [ 0.0835,  0.2095, -0.4398, -0.0050, -0.1596],
                      [ 0.3616, -0.4206,  0.0112, -0.4096, -0.3411]], device='cuda:0')),
             ('fc2.bias',
              tensor([-0.1562,  0.4329,  0.1964], device='cuda:0')),
             ('fc3.weight',
              tensor([[ 0.2798,  0.1290, -0.4579]], device='cuda:0')),
             ('fc3.bias', tensor([-0.1769], device='cuda:0'))])